In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Esta função cria séries temporais univariadas
# Devolve batch_size séries, cada uma com tamanho n_steps 
# Como é uma série univariada, apenas existe 1 valor em cada timestep: (Batch, Timestep, 1) 

def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + noise
    return series[..., np.newaxis].astype(np.float32)

In [ ]:
# Criar os conjuntos de treino, validação e teste
# Cada série temporal tem 50 passos

np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [ ]:
#Qual o formato dos tensores criados para cada um dos conjuntos?
# Treino, Validação e Teste



In [ ]:
# Criar uma função para visualizar alguns exemplos de séries temporais

def plot_series(series, y=None, y_pred=None, x_label="$t$", y_label="$x(t)$"):
    plt.plot(series, ".-")
    if y is not None:
        plt.plot(n_steps, y, "bx", markersize=10)
    if y_pred is not None:
        plt.plot(n_steps, y_pred, "ro")
    plt.grid(True)
    if x_label:
        plt.xlabel(x_label, fontsize=16)
    if y_label:
        plt.ylabel(y_label, fontsize=16, rotation=0)
    plt.hlines(0, 0, 100, linewidth=1)
    plt.axis([0, n_steps + 1, -1, 1])

In [ ]:
# Mostrar exemplos de 3 séries temporais
# O X marca o valor a prever em cada caso

fig, axes = plt.subplots(nrows=1, ncols=3, sharey=True, figsize=(12, 4))
for col in range(3):
    plt.sca(axes[col])
    plot_series(X_valid[col, :, 0], y_valid[col, 0],
                y_label=("$x(t)$" if col==0 else None))
save_fig("time_series_plot")
plt.show()

In [ ]:
# Baseline 1: O preditor devolve o último valor da série temporal
# Nem sequer é preciso nenhum modelo para este cálculo. Basta devolver o último valor da sequência
# Vamos utilizar apenas os dados reservados para teste
y_pred = X_test[:, -1]

# Determinar o menor e maior erro em cada uma das series de teste
difference = np.absolute(y_pred - y_test)
print('Min: ', np.min(difference))
print('Max: ', np.max(difference))


In [ ]:
# Cálculo do erro obtido por esta abordagem no conjunto de teste (MSE) 

np.mean(keras.losses.mean_squared_error(y_test, y_pred))

In [ ]:
# Visualização do erro na primeira série de teste
# O circulo vermelho assinala o valor previsto e o X azul o valor correto

# Experimentar visualizar para outras séries de teste
serie = 0

plot_series(X_test[serie, :, 0], y_test[serie, 0], y_pred[serie, 0])
plt.show()

In [ ]:
# Baseline 2: A previsão é uma combinação linear dos inputs
# Criar uma rede neuronal simples com uma única camada 
# Os pesos correspondem aos parâmetros da combinação linear

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# A entrada é linearizada com uma camada Flatten
# A saída recebe as contribuições pesadas 

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])


In [ ]:
model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

In [ ]:
# Avaliação no conjunto de teste

# Qual das 2 abordagens baseline obteve melhores resultados?

model.evaluate(X_test, y_test)


In [ ]:
# Visualizar os resultados
# Visualização da evolução da loss

import pandas as pd 
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()


In [ ]:
# Visualização do erro na primeira série de teste
# O circulo vermelho assinala o valor previsto e o X azul o valor correto

# Experimentar visualizar para outras séries de teste
serie = 0
y_pred = model.predict(X_valid)
plot_series(X_valid[serie, :, 0], y_valid[serie, 0], y_pred[serie, 0])
plt.show()

In [ ]:
weights, bias = model.layers[1].get_weights()
weights = np.absolute(weights)

In [ ]:
# Que interpretação é que faz da informação apresentada neste gráfico?

plt.plot(weights)

In [ ]:
# Já existem valores baseline baseados em abordagens muito simples
# Agora passamos para a implementação de RNN, para verificar qual a melhoria de desempenho

# https://www.tensorflow.org/guide/keras/rnn
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/SimpleRNN

# Abordagem RNN 1: Utilizar células RNN simples

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)


# Modelo apenas com 1 camada e 1 célula recorrente
# Não é necessário indicar o tamanho da sequência de input (número de steps)

modelR = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])



In [ ]:
# Quantos parâmetros tem este modelo modelR?
# Para que servem?



In [ ]:
optimizer = keras.optimizers.Adam(lr=0.005)

modelR.compile(loss="mse", optimizer=optimizer)
history = modelR.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

In [ ]:
# Avaliar o desempenho no conjunto de teste
modelR.evaluate(X_test, y_test)


In [ ]:
# Visualizar os resultados
# Visualização da evolução da loss

import pandas as pd 
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()


In [ ]:
# Analise o desempenho e efetue a comparação com as abordagens baseline


In [ ]:
# Abordagem RNN 2: Utilizar células RNN simples, 
# Colocar várias camadas para aumentar a complexidade do modelo - Deep RNN 

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# Dois detalhes importantes
# 1. Parâmetro return_sequences indica se é devolvida toda a sequência ou apenas o último valor
# 2. A camada de saída é um nó Dense normal. Não tem que ser recorrente porque queremos apenas prever um valor
modelR2 = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])


In [ ]:
# Explicar o número de parâmetros do modelo modelR2
modelR2.summary()

In [ ]:
modelR2.compile(loss="mse", optimizer="adam")

history = modelR2.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

In [ ]:
# Avaliar o desempenho no conjunto de teste
modelR2.evaluate(X_test, y_test)


In [ ]:
# Abordagem 3: Substituir as células RNN simples por LSTM
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

modelR3 = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20),
    keras.layers.Dense(1)
])



In [ ]:
# Aumento muito significativo do número de parâmetros
# O treino vai demorar mais

modelR3.summary()

In [ ]:
modelR3.compile(loss="mse", optimizer="adam")

history = modelR3.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

In [ ]:
# Avaliar o desempenho no conjunto de teste
modelR3.evaluate(X_test, y_test)

In [ ]:
# Visualizar os resultados
# Visualização da evolução da loss

import pandas as pd 
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()

In [ ]:
# Substituir as células LSTM por células GRU e analisar eventuais variações nos resultados obtidos

# Poderão igualmente ser testadas diferentes alterações na arquitetura das RNN ou nos híper-parâmetros definidos


In [ ]:
# Proposta de Trabalho

# É possível combinar camadas CNN e camadas RNN numa rede
# No contexto de séries temporais, as camadas 1D CNN servem para fazer operações de convolução a nível temporal
# Utilizar 1D CNN para fazer o downsampling da sequência e para criar vários filtros
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D




In [ ]:
# Qual é o tamanho de cada série temporal que sai da camada convolucional?

In [ ]:
# Explicar o número de pesos de cada camada 
